In [ ]:
# !pip install transformers

In [ ]:
from transformers import RobertaTokenizerFast,RobertaConfig, RobertaForMaskedLM
from pathlib import Path
import os
from tokenizers import ByteLevelBPETokenizer
# from tokenizer import *
from tqdm.auto import tqdm
from pathlib import Path
import torch
import numpy as np

In [ ]:
# import all text files from data folder

if not os.path.exists('/content/data'):
    os.mkdir('/content/data')

paths = [str(f) for f in Path(r"/content/data").glob('*.txt')]

In [ ]:
tokenizer = ByteLevelBPETokenizer()
# train the model
tokenizer.train(files=paths, vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>',
                                '</s>', '<unk>', '<mask>'])

In [ ]:
# saves token as txt and json
if not os.path.exists('/content/tokens'):
    os.mkdir('/content/tokens')

tokenizer.save_model('/content/tokens')

['/content/tokens/vocab.json', '/content/tokens/merges.txt']

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("/content/tokens")
tokenizer('Abraham!')


file /content/tokens/config.json not found
file /content/tokens/config.json not found


{'input_ids': [0, 21073, 5, 2], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
tokenizer("sin").input_ids

[0, 27965, 2]

In [ ]:
input_ids = tokenizer("sin you die").input_ids

tensor([0.6775, 0.1839, 0.9944, 0.4744, 0.3151])
0
[]
1
[]
2
[]
3
[]
4
[]


In [ ]:
for p in tqdm(paths):
    with open(p, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    batch = tokenizer(lines, max_length=512,
                      padding='max_length', truncation=True)
len(batch)

  0%|          | 0/8 [00:00<?, ?it/s]

2

In [ ]:
labels = torch.tensor([v for k, v in batch.items()][0])
mask = torch.tensor([v for k, v in batch.items()][1])

In [ ]:
print(labels)

tensor([[  0, 452, 399,  ...,   1,   1,   1],
        [  0,   6, 452,  ...,   1,   1,   1],
        [  0,   6, 452,  ...,   1,   1,   1],
        ...,
        [  0,   6, 452,  ...,   1,   1,   1],
        [  0, 452, 399,  ...,   1,   1,   1],
        [  0,   2,   1,  ...,   1,   1,   1]])


In [ ]:

# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3

In [ ]:
input_ids.shape

torch.Size([31103, 512])

In [ ]:
input_ids[0][:100]

tensor([    0,   452,   399,   470,   338,     3,   281,     3,     3,     3,
          644,     3,    30,     3,  1092,   264,  2322,   277,   438,   399,
        12232,   264,  1571,   273,   264,   778,     3,   815,    31,   225,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1])

In [ ]:
encodings = {'input_ids': input_ids,
             'attention_mask': mask,
             'labels': labels}

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key,
                tensor in self.encodings.items()}


In [ ]:
dataset = Dataset(encodings)

In [ ]:
loader = torch.utils.data.DataLoader(
    dataset, batch_size=16, shuffle=True)

In [ ]:
tokenizer.vocab_size

30522

In [ ]:
config = RobertaConfig(
    vocab_size= tokenizer.vocab_size,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [ ]:
# initialize the RoBERTa model with a language modeling (LM) head.
model = RobertaForMaskedLM(config)